### Aryl Halide Clustering
#### 1. Process dataset and remove highly correlated features (R2 > 0.95)
#### 2. Compare PCA and UMAP for dimensionality reduction and clustering
#### 3. Make optimal clusters and visualize molecules closest to cluster centers

In [ ]:
# package imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from rdkit import Chem
from rdkit.Chem import Draw

# scikit learn!
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import pairwise_distances_argmin_min
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata
from scipy.spatial.distance import cdist

# umap for dimensionality reduction
import umap

# nice plotting
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns; sns.set()

In [ ]:
# cleaned aryl X dataset
# We kept parameters for the low energy conformer (removing those for min/max and Bolztmann average, which are highly correlated)
arylx = pd.read_csv('arylx.csv')
arylx.head()

In [ ]:
# the molecular descriptors
X = arylx.select_dtypes(include=['number'])

# Feature scaling through standardization (or Z-score normalization) is an important preprocessing step 
# for many machine learning algorithms. Standardization involves rescaling the features such that they 
# have the properties of a standard normal distribution with a mean of zero and a standard deviation of one. 

X_scaled=pd.DataFrame(scale(X),index=X.index, columns=X.columns)

# drop zero-variance features
zero_std_cols = X_scaled.columns[X_scaled.std() == 0]
X_scaled=X_scaled[X_scaled.columns.difference(zero_std_cols)]
print (f"Dropping {len(zero_std_cols)} features {zero_std_cols}")

# drop highly correlated features
corr = X_scaled.corr().abs()
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
X_scaled = X_scaled.drop(to_drop, axis=1)
print (f"Dropping {len(to_drop)} features {to_drop}")

In [ ]:
X_scaled.head()

#### At this point we have 22 descriptors remaining

In [ ]:
# define the dimensionalities of the reduced representation to study
# We will compare the performance of clustering based on reduced dimensionality representations of size 3 and 2 dimensions
# Because tSNE cannot convert high dimension data, we only use size 3 and 3 dimensions
dims = [3, 2]

# dictionary to store data at different levels of dimensionality reduction
dfs={}

# UMAP
np.random.seed(0)
n_neighbors = int(np.sqrt(X_scaled.shape[1]))
for dim in dims:
    key = f"umap{dim}"
    dfs[key] = pd.DataFrame(umap.UMAP(n_components=dim, n_neighbors=n_neighbors, random_state=np.random.RandomState(0)).fit_transform(X_scaled),
    index=X_scaled.index)

# PCA
for dim in dims:
    pc = pd.DataFrame(PCA(n_components=dim).fit_transform(X_scaled), index=X_scaled.index)
    key = f"pc{dim}"
    dfs[key] = pc.iloc[:, :dim]

# tSNE
for dim in dims:
    tsne = pd.DataFrame(TSNE(n_components=dim, max_iter=1000, random_state=0).fit_transform(X_scaled), index=X_scaled.index)
    key = f"tsne{dim}"
    dfs[key] = tsne.iloc[:, :dim]

# The user warnings are not concerning - relate to running calculations on 1 vs. several processors

In [ ]:
# Visualize the embeddings
f, ax = plt.subplots(1, 3, figsize=(15, 5))  # , dpi=200)

dfs["pc2"].columns = ["PC1", "PC2"]
dfs["umap2"].rename(columns={0: "UMAP1", 1: "UMAP2"}, inplace=True)
dfs["tsne2"].columns = ["tSNE1", "tSNE2"]

sns.scatterplot(
    x="PC1",
    y="PC2",
    data=dfs["pc2"],
    s=15,
    alpha=0.5,
    linewidth=0.25,
    edgecolor="face",
    ax=ax[0],
).set_title("PC projection")
sns.scatterplot(
    x="UMAP1",
    y="UMAP2",
    data=dfs["umap2"],
    s=15,
    alpha=0.5,
    linewidth=0.25,
    edgecolor="face",
    ax=ax[1],
).set_title("UMAP projection")
sns.scatterplot(
    x="tSNE1",
    y="tSNE2",
    data=dfs["tsne2"],
    s=15,
    alpha=0.5,
    linewidth=0.25,
    edgecolor="face",
    ax=ax[2],
).set_title("tSNE projection")

In [ ]:
# run another UMAP, compare the result with the first run
n_neighbors = int(np.sqrt(X_scaled.shape[1]))
print(n_neighbors)
for dim in dims:
    key = f"umap_rerun_{dim}"
    dfs[key] = pd.DataFrame(umap.UMAP(n_components=dim, n_neighbors=n_neighbors, random_state=0).fit_transform(X_scaled),
    index=X_scaled.index)

In [ ]:
dfs

In [ ]:
# Visualize the embeddings
f, ax = plt.subplots(1, 3, figsize=(15, 5)) #, dpi=200)

dfs['pc2'].columns = ['PC1', 'PC2']
dfs['umap2'].rename(columns={0:'UMAP1', 1:'UMAP2'}, inplace=True)
dfs['tsne2'].columns = ['tSNE1', 'tSNE2']

sns.scatterplot(x='PC1', y='PC2', data=dfs['pc2'], s=15, alpha=0.5, linewidth=0.25, edgecolor='face', ax=ax[0]).set_title("PC projection")
sns.scatterplot(x='UMAP1', y='UMAP2', data=dfs['umap2'], s=15, alpha=0.5, linewidth=0.25, edgecolor='face', ax=ax[1]).set_title("UMAP projection")
sns.scatterplot(x='tSNE1', y='tSNE2', data=dfs['tsne2'], s=15, alpha=0.5, linewidth=0.25, edgecolor='face', ax=ax[2]).set_title("tSNE projection")

In [ ]:
# Testing the quality of clustering approaches
# Here we compare the effect of different dimensionality representations and the number of clusters
# The sillhouette score quantifies the intra-cluster distance vs. inter-cluster distance.

# Define the numbeor of clusters to study
N_CLS_list = list(range(10, 30))

def silhouette_scores_hierarchical(data, n_cls_list):
    """helper function to compute a silhouette score for hierarchical clustering using Ward linkage"""
    z = linkage(data, method='ward')
    result = pd.Series(index=n_cls_list, dtype=float)
    for n_cls in n_cls_list:
        cls = fcluster(z, n_cls, criterion='maxclust')
        result.loc[n_cls] = silhouette_score(data, cls)
    return result

# populate silhouette scores for all number of clusters and all dimensionality reductions that are pre-calculated
silh_scores = pd.DataFrame(index=N_CLS_list)

for key, value in dfs.items():
    silh_scores[key] = silhouette_scores_hierarchical(value, N_CLS_list)

# plot the silhouette scores
silh_scores.plot(xlabel='number of clusters',ylabel='silhouette score')

In [ ]:
# Testing the quality of clustering approaches using different scoring methods
# DBCV ref: Moulavi, Davoud, et al. "Density-based clustering validation." Proceedings of the 2014 SIAM International Conference on Data Mining. Society for Industrial and Applied Mathematics, 2014.
# To install dbcv: python -m pip install "git+https://github.com/FelSiq/DBCV"
import dbcv

# Define the numbeor of clusters to study
N_CLS_list = list(range(10, 30))

def dbcv_scores_hierarchical(data, n_cls_list):
    """helper function to compute a dbcv score for hierarchical clustering using Ward linkage"""
    z = linkage(data, method='ward')
    result = pd.Series(index=n_cls_list, dtype=float)
    for n_cls in n_cls_list:
        cls = fcluster(z, n_cls, criterion='maxclust')
        result.loc[n_cls] = dbcv.dbcv(data, cls)
    return result

# populate dbcv scores for all number of clusters and all dimensionality reductions that are pre-calculated
dbcv_scores = pd.DataFrame(index=N_CLS_list)

for key, value in dfs.items():
    dbcv_scores[key] = dbcv_scores_hierarchical(value, N_CLS_list)

# plot the dbcv scores
dbcv_scores.plot(xlabel='number of clusters',ylabel='DBCV score')

In [9]:
# From the plots above:
# UMAP leads to better quality clusters compared to PCA (higher sillhouette scores overall)
# For the UMAP results, using 5, 10 or 20 dimensions gives similar performance
# for lower cluster numbers, (10-12), 2 dimensions works best

In [ ]:
# final number of clusters to produce (in the case of UMAP)
NCLS = 18
features='umap2'

# linkage and clustering for selected featurization
z = linkage(dfs[features], method="ward")
cls = fcluster(z, NCLS, criterion='maxclust')

# plot clustering
plt.figure(figsize=(8, 8))
sns.scatterplot(x="UMAP1", y="UMAP2", data=dfs['umap2'], s=35, alpha=0.7, linewidth=0.25, edgecolor='face',
palette='rainbow_r', legend='full', hue=cls).set_title("10 UMAPs - 2-UMAP projection")
_=plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# final number of clusters to produce (in the case of PCA)
NCLS = 11
features='pc2'

# linkage and clustering for selected featurization
z = linkage(dfs[features], method="ward")
cls = fcluster(z, NCLS, criterion='maxclust')

# plot clustering
plt.figure(figsize=(8, 8))
sns.scatterplot(x="PC1", y="PC2", data=dfs['pc2'], s=35, alpha=0.7, linewidth=0.25, edgecolor='face',
palette='rainbow_r', legend='full', hue=cls).set_title("10 PCs - 2-PC projection")
_=plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# final number of clusters to produce (in the case of tSNE)
NCLS = 15
features='tsne2'


# linkage and clustering for selected featurization
z = linkage(dfs[features], method="ward")
cls = fcluster(z, NCLS, criterion='maxclust')

# plot clustering
plt.figure(figsize=(8, 8))
sns.scatterplot(x="tSNE1", y="tSNE2", data=dfs[features], s=35, alpha=0.7, linewidth=0.25, edgecolor='face',
palette='rainbow_r', legend='full', hue=cls).set_title("10 tSNE - 2-tSNE projection")
_=plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

In [ ]:
# build rdkit molecules for all candidates
mols = pd.Series([Chem.MolFromSmiles(smi) for smi in arylx.smiles]).to_frame('mol')

# How many central molecules to display?
n_per_cluster = 5

# store central candidates for
cands=[]
for group, data in mols.groupby(cls):
   
    # get descriptor data for this cluster
    print (f"Cluster {group}, n molecules: {len(data)}")
   
    desc_data=dfs[features].loc[data.index]
    
    # compute distances of these molecules to their center
    dists=pd.Series(cdist([desc_data.mean()], desc_data)[0],
    index=desc_data.index)
    
    # select top n central molecules
    selected=dists.sort_values().head(n_per_cluster).index
    smi=mols.loc[selected]['mol'].map(Chem.MolToSmiles)
    smi=smi.reset_index(drop=True).to_frame(f"Cluster{group}")
    cands.append(smi)
    
    ms = data['mol'].loc[selected]
    display(Draw.MolsToGridImage(ms, molsPerRow=n_per_cluster))
    img = Draw.MolsToGridImage(ms, molsPerRow=n_per_cluster)

    png = img.data
    with open('./cluster_'+str(group)+'.png','wb+') as outf:
        outf.write(png)
cands = pd.concat(cands, axis=1)



In [14]:

# make a handful of umaps with various random seeds
umaps = []
for i in range(50):
    umaps.append(pd.DataFrame(umap.UMAP(n_components=10, n_neighbors=n_neighbors).fit_transform(X_scaled),
                              index=X_scaled.index))

        

In [ ]:
# add the original umap that was used for clustering
umaps = [dfs['umap2']] + umaps

from scipy.spatial.distance import pdist, squareform

def sample_dist(data):
    dists = pdist(data)
    return [ min(dists), np.mean(dists), max(dists)]

def kenStone(X, k, metric='euclidean'):

    # safety checks
    assert isinstance(k, int)
    assert k >= 2
    assert k <= X.shape[0]
    
    # distance matrix
    d = squareform(pdist(X, metric))
    
    # seed pick the pair that's furthest apart
    selected = list(np.unravel_index(np.argmax(d), d.shape))

    while len(selected) < k:
        # add sample whose minimum distance to the selected samples is largest

        selected.append(np.argmax(d[selected,].min(axis=0)))
    
    return selected

# selection of dataset to use
dat = umaps[7]
NCLS=15

ret_cls = []

for dat in umaps:
    # clustering
    z = linkage(dat, method="ward")
    cls = fcluster(z, NCLS, criterion='maxclust')

    selected = []
    for group, d in dat.groupby(cls):
        # compute distances of these molecules to their center
        dists=pd.Series(cdist([d.mean()], d)[0], index=d.index)

        # select top n central molecules
        selected.append(dists.sort_values().index[0])
        
    dcls = sample_dist(dat.loc[selected])
    ret_cls.append(dcls)

ret_cls = pd.DataFrame(ret_cls, columns=[ 'clustering min. dist', 'clustering avg. dist', 'clustering max. dist'])

ret_ks = []
for dat in umaps:
    ks = kenStone(dat, NCLS)
    dks = sample_dist(dat.iloc[ks])
    ret_ks.append(dks)
ret_ks = pd.DataFrame(ret_ks, columns=[ 'ks min dist', 'ks avg dist', 'ks max dist'])

# random sampling
ret_rnd = []
for dat in umaps:
    for i in range(100):
        ret_rnd.append(sample_dist(dat.sample(NCLS)))

ret_rnd = pd.DataFrame(ret_rnd, columns=[ 'random min. dist', 'random avg. dist', 'random max. dist'])

f, ax = plt.subplots(3,1, figsize=(6, 8))
ret_cls.plot(kind='hist', histtype='step', facecolor='#008000', edgecolor='k' , fill=True,
             subplots=True, ax=ax, color='black', bins=45, xlim=(-1, 20), density=False)
'''ret_rnd.plot(kind='hist', histtype='step', color='#0000C0', linewidth=2, linestyle='--',
             subplots=True, ax=ax, bins=45, xlim=(-1, 16), density=True)'''
ax[2].legend(loc='upper left')
ax[0].set_ylabel('Frequency')
ax[1].set_ylabel('Frequency')
ax[2].set_ylabel('Frequency')
